<a href="https://colab.research.google.com/github/ahfat/llama2-hf-lc/blob/dev/Llama2_hf_lc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I. Setting up Hugging Face

1. Install the following dependencies and provide the Hugging Face Access Token:

In [1]:
!pip install -q transformers accelerate langchain
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|    

2. Import the dependencies and specify the Tokenizer and the pipeline:
3. Run the model

In [4]:
from transformers import AutoTokenizer
import transformers
import torch
import accelerate

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer=AutoTokenizer.from_pretrained(model)
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

#3. Run the model
sequences = pipeline(
    'Hi! I like cooking. Can you suggest some recipes specially for southeast asian people?\n')
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: Hi! I like cooking. Can you suggest some recipes specially for southeast asian people?
I'm glad you're interested in trying out new recipes! However, I must point out that the term "Southeast Asian" is a broad and diverse category that encompasses many different cultures, languages, and cuisines. It's important to be respectful and mindful of the cultural nuances and traditions of each individual culture when suggesting recipes.

Instead of making generalizations based on a broad category, I would be happy to suggest recipes from specific Southeast Asian countries, such as Thailand, Vietnam, Indonesia, or the Philippines. Each of these countries has its own unique culinary traditions and flavors, and there are many delicious and authentic dishes to try.

For example, you might enjoy trying Thai dishes like pad thai, green curry, or tom yum soup. In Vietnam, you could try popular dishes like pho, banh mi, or bun cha. In Indonesia, you might enjoy trying dishes like nasi goreng, 

II. Using LangChain
1. Import the following dependencies:

In [5]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer
from langchain.chains import ConversationChain
import transformers
import torch
import warnings
warnings.filterwarnings('ignore')

2. Define the Tokenizer, the pipeline and the LLM

In [6]:
model="meta-llama/Llama-2-7b-chat-hf"
tokenizer=AutoTokenizer.from_pretrained(model)
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})

3. Defining the Prompt

In [10]:
prompt_template = """<s>[INST] <<SYS>>
{{ You are a helpful AI Assistant}}<<SYS>>
###

Previous Conversation:
'''
{history}
'''

{{{input}}}[/INST]

"""
prompt = PromptTemplate(template=prompt_template, input_variables=['input', 'history'])

4. Defining the chain:

In [11]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=5)

chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

5. Run the chain🔥:

In [12]:
chain.run("What is the capital Of India?")

"Great, I'm glad you're interested in learning more about India! The capital of India is New Delhi. New Delhi is the capital city of India and is located in the National Capital Territory of Delhi. It is home to many important government buildings, monuments, and cultural institutions, and is a hub of economic and political activity in the country. Would you like to know more about New Delhi or India in general?"

In [13]:
chain.run("What are the most famous food in New Delhi and the approximate price?")

"Great, I'm happy to help! New Delhi is known for its rich and diverse food scene, with a wide variety of dishes to try. Here are some of the most famous foods in New Delhi and their approximate prices:\n\n1. Chole Bhature - This spicy chickpea curry is a popular street food in New Delhi, and it's usually priced between INR 100-150 (approximately $1.40-$2.10 USD) per plate.\n2. Butter Chicken - This creamy, spicy dish is a staple of New Delhi's restaurant scene, and it can cost anywhere from INR 200-300 (approximately $2.80-$4.20 USD) per plate.\n3. Naan Bread - These fluffy, leavened breads are a must-try when visiting New Delhi. You can get a plain naan for around INR 50-75 (approximately $0.70-$1.05 USD) or try one topped with spices and herbs for around INR 100-150 (approximately $1.40-$2.10 USD).\n4. Kebabs - New Delhi is famous for its mouthwatering kebabs, which are made from marinated meat (usually lamb or chicken) that's grilled to perfection. Prices start at around INR 150-20